#  Etapa de entrenamiento y testeo de un modelo de análisis de sentimiento

In [ ]:
#Librerias: 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
#Cargando el dataset creado:
df = pd.read_csv('final.csv')

In [ ]:
df.head()

,overall,reviewText
0,4,"['pretty', 'good', 'game', 'daughter', 'love',..."
1,5,"['first', 'let', 'clear', 'one', 'thing', 'rem..."
2,1,"['blue', 'button', 'stopped', 'working', 'quic..."
3,2,"['never', 'played', 'game', 'sc', 'series', 't..."
4,2,"['no', 'no']"


In [ ]:
#Asignar clasificaciones de estrellas a etiquetas de sentimiento (0 = negativo, 1 = positivo):
etiqueta = {1: 0, 2: 0, 4: 1, 5: 1}
df['etiqueta'] = df['overall'].map(etiqueta)

X = df['reviewText']
y = df['etiqueta']

#Dividir los datos en entrenamiento y prueba

In [ ]:
#Dividir los datos en datos de entrenamiento (80%) y datos de prueba (20%):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Frecuencia de término-Frecuencia de documento inversa

Explicación de la elección de parámetros para tfidfVectorizer:


* min_df: establezca min_df en un valor más alto para excluir algunas de las palabras menos frecuentes y reducir potencialmente el ruido en los datos
* max_df: dado que hemos visto que la revisión promedio es bastante detallada, decidimos evitar el uso de palabras que podrían considerarse palabras vacías específicas del contexto.
* ngram_range: hemos configurado esta función en una serie de valores diferentes de hasta 5 y la hemos probado en un modelo básico. Este valor parece ser el más adecuado.
* max_feature: con una cardinalidad de más de 40k, este parámetro tuvo que establecerse bastante alto. Nos decidimos por 2000 ya que tuvo un mejor resultado que en 500 (81% F1-score) y 1000 (83% F1-score). Establecerlo más alto parecía crear demasiada demanda computacional y también podría conducir a un sobreajuste.



In [ ]:
vectorizer = TfidfVectorizer(min_df=50, max_df=0.90, ngram_range=(1,4), max_features=2000)
vectorizer.fit(X) 
X_train_tf = vectorizer.transform(X_train)  
#Tomando X como entrada y convirtiéndolo en matriz de características (valores numéricos):
X_train_tf = X_train_tf.todense()
X_test_tf = vectorizer.transform(X_test)  
#Tomando X como entrada y convirtiéndolo en matriz de características (valores numéricos):
X_test_tf = X_test_tf.todense()

In [ ]:
X_train_tf = np.asarray(X_train_tf) 
X_test_tf = np.asarray(X_test_tf)

# Primer modelo: SVC

In [ ]:
#Crear una instancia del modelo SVM con hiperparámetros básicos:
svc = SVC(kernel='linear', C=1, random_state=42)

#Entrene el modelo SVM en el conjunto de entrenamiento:
svc.fit(X_train_tf, y_train)

#Use el modelo SVM entrenado para predecir el sentimiento del conjunto de prueba:
y_pred = svc.predict(X_test_tf)

#Evalúe el rendimiento del modelo SVM utilizando accuracy, precision, recall y F1-score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)

Accuracy: 0.8435
Precision: 0.8508728179551123
Recall: 0.839153959665519
F1-score: 0.844972758791481


In [ ]:
#Creación de una función para encontrar los mejores parámetros:
def svc_tuning(X_train, y_train, X_test, y_test):
    #Establecer los parámetros para la búsqueda de cuadrícula:
    param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['poly', 'rbf', 'sigmoid', 'linear']}
    
    #Crear el clasificador SVM:
    svc = SVC()
    
    #Creano el GridSearchCV:
    grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    
    #Ajuste del GridSearchCV a los datos de entrenamiento:
    grid_search.fit(X_train, y_train)
    
    #Use el mejor modelo de la búsqueda de cuadrícula para hacer predicciones sobre los datos de prueba:
    y_pred = grid_search.predict(X_test)
    
    #Calculando el accuracy, precision, recall y F1-score:
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print('Best parameters:', grid_search.best_params_)
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1-score:', f1)

In [ ]:
svc_tuning(X_train_tf[0:2000], y_train[0:2000], X_test_tf, y_test)

Best parameters: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.81525
Precision: 0.8355809128630706
Recall: 0.7924249877029022
F1-score: 0.8134309517798536


In [ ]:
#Crear una instancia del modelo SVM con los mejores hiperparámetros:
svc = SVC(kernel='rbf', C=1, gamma=1, random_state=42)

#Entrene el modelo SVM en el conjunto de entrenamiento:
svc.fit(X_train_tf, y_train)

#Use el modelo SVM entrenado para predecir el sentimiento del conjunto de prueba:
y_pred = svc.predict(X_test_tf)

#Evaluar el rendimiento del modelo SVM usando accuracy, precision, recall, and F1-score:
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)

Accuracy: 0.85275
Precision: 0.8553149606299213
Recall: 0.8548942449581899
F1-score: 0.8551045510455104


# Segundo modelo: XGBoost

In [ ]:
#Definir el modelo XGBoost:
xgb_model = XGBClassifier()

#Parametros:
params = {
    "learning_rate": [0.1, 0.01, 0.001],
    "max_depth": [3, 5, 7],
    "subsample": [0.5, 0.8, 1.0],
    "colsample_bytree": [0.5, 0.8, 1.0],
    "gamma": [0.01, 0.1, 1.0]
}

#GridSearchCV:
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=3
)

#Ajuste la búsqueda de cuadrícula a los datos de entrenamiento:
grid_search.fit(X_train_tf[:2000], y_train[:2000])

#Imprime los mejores parámetros y la puntuación correspondiente:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


In [ ]:
#Modelo XGBoost:
xgb_model = XGBClassifier(
    n_estimators=1000,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.5,
    gamma = 0.1, 
    random_state=42
)

#Entrenar modelo XGBoost:
xgb_model.fit(X_train_tf, y_train)

#Evaluar modelo XGBoost:
y_pred = xgb_model.predict(X_test_tf)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
#Resultado:
print(accuracy)

0.853
